In [ ]:
!pip install pyLDAvis

In [152]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [ ]:
!python3 -m spacy download en

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [ ]:
df = pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

In [ ]:
data = df.content.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

In [ ]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

In [10]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

In [ ]:
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

In [ ]:
#Этот блок выдает ошибку "CalledProcessError"
mallet_path = r'C:\Users\Иришка\Downloads\mallet-2.0.8' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

In [13]:
def best_group():
  maxim = 0
  number = 0
  for i in range(10,30):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=i, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=5,
                                           alpha='auto',
                                           per_word_topics=True)
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    coherence_lda = coherence_model_lda.get_coherence()
    if coherence_lda >= maxim:
      maxim = coherence_lda
      number = i
    return number

In [ ]:
best_group()

In [15]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=5,
                                           alpha='auto',
                                           per_word_topics=True)

In [16]:
topics = lda_model.show_topics(10, formatted=False)
#print(topics[0][1][1])

In [17]:
from collections import Counter

In [19]:
text_in_topic = {}
n=0
for text in texts:
  n+=1
  count = Counter()
  for word in text:
    for topic in topics:
      id = topic[0]
      for i in range(len(topic[1])):
        if topic[1][i][0] == word:
          count[id]+= topic[1][i][1]
  d = dict(count.most_common())
  max = 0
  i_cor = 0
  for i in d:
    if d.get(i) > max:
      max = d.get(i)
      i_cor = i
    text_in_topic.update({n : i_cor})
#print(text_in_topic[0])

In [20]:
groups = {}
for i in range(0,10):
  lis = []
  for key, value in text_in_topic.items():
    if value == i:
      lis.append(key)
    groups.update({i:lis})

In [21]:
def computeTF(t):
  numOfWords = dict.fromkeys(t, 0)
  for word in t:
    numOfWords[word] += 1
  tfDict = {}
  textCount = len(t)
  for word, count in numOfWords.items():
    tfDict[word] = count / float(textCount)
  return tfDict

In [ ]:
tfs = {}
for i in range (0,10):
  num_text = groups[i]
  for n in num_text:
    need_text = texts[n-1]
    get_tf = computeTF(need_text) #список словарей (для каждого текста словарь со значениями)
    tfs.update({i: [n , get_tf]})
print(tfs[0])

In [23]:
import math

In [24]:
def computeIDF(documents):
    N = len(documents[0])
    idfDict = dict.fromkeys(documents[0], 0)
    for document in documents: 
        for word, val in document.items():
          if val > 0:
            if word in idfDict:
                idfDict[word] += 1
            else:
              idfDict.update({word:1})
    for word, val in idfDict.items():
        idfDict[word] = math.log(N / float(val))
    return idfDict

In [ ]:
idfs = {}
for i in range(0,10):
  num_tex = groups[i]
  for n in num_tex:
    ls = []
    new = []
    need_tex = texts[n-1]
    numOfWords_ = dict.fromkeys(need_tex, 0)
    for word in need_tex:
      numOfWords_[word] += 1
    ls.append(numOfWords_)
    #print(len(ls[0]))
  com_idf = computeIDF(ls)
  idfs.update({i: com_idf})
#print(idfs)

In [ ]:
from collections import Counter

In [149]:
all_tf_idf = []
for i in range (0,10):
  num = groups[i]
  ttfidf = []
  for n in num:
    need = texts[n-1]
    tf_ = tfs[i][1]
    idf_ = idfs[i]
    sor_words = []
    words = Counter()
    for word in need:
      if word in tf_ and word in idf_:
        tf = tf_[word]
        idf = idfs[i][word]
        tfidf = tf*idf
        #print(i,n,word,tfidf)
        words.update({word:tfidf})
    d = dict(words.most_common(5))
    sor_words.append(d)
    #print(sor_words[0])
    for w in sor_words:
      #print(w,n)
      for key, value in w.items():
        charact = [key , value, n, i]
        all_tf_idf.append(charact)
#print(all_tf_idf)

In [150]:
df = pd.DataFrame(all_tf_idf)
df.columns = ['word','tf_idf', 'text_number', 'topic']
df

,word,tf_idf,text_number,topic
0,year,1.028131,18,0
1,make,0.411252,18,0
2,get,0.370127,18,0
3,would,0.370127,18,0
4,good,0.287877,18,0
...,...,...,...,...
42884,enviroleague,1.337965,11277,9
42885,youth,1.057157,11277,9
42886,program,1.057157,11277,9
42887,organization,0.809386,11277,9


Coherence score - это показатель, который измеряет оценку одной темы (topic) путем оценки степени семантического сходства между словами в этой теме, обладающими наибольшими весами.